# Get Paragraph vectors

In [5]:
import gensim
from gensim.models import Doc2Vec, KeyedVectors
import logging
import time
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import requests
import matplotlib.pyplot as plt
import gensim.downloader as api # Downloading the Google pretrained Word2Vec Model

In [6]:
import numpy as np
import requests
import random
from tqdm.notebook import tqdm
tqdm.pandas()

In [34]:
import os
import json
import glob
import re
import sys
import collections
from nltk import flatten
import dask
from dask import delayed,compute
import dask.dataframe as dd
from dask.multiprocessing import get
import logging
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [75]:
#Read dataset
temp_df = pd.read_csv('unique_books.csv')

In [76]:
temp_df.head()

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
0,0,2,2318,3,41929,398,998.0,671015206,9.780671e+12,The Millionaire Next Door: The Surprising Secr...,4.00,43937.0,"['Thomas J. Stanley', 'William D. Danko']",The incredible national bestseller that is cha...,9.780670e+12,The Millionaire Next Door: The Surprising Secr...,"['Economics', 'Finance', 'Nonfiction', 'Busine...",the incredible national bestseller ...
1,389,2,26,4,41929,227,968.0,307277674,9.780307e+12,The Da Vinci Code,3.79,1447148.0,['Dan Brown'],An ingenious code hidden in the works of Leona...,9.780310e+12,The Da Vinci Code,"['Fiction', 'Mystery', 'Thriller']",an ingenious code hidden in t...
2,13461,2,315,3,41929,193,4894.0,91883768,9.780092e+12,Who Moved My Cheese?,3.74,237272.0,"['Spencer Johnson', 'Kenneth H. Blanchard']","Written by Spencer Johnson, coauthor of The On...",9.780090e+12,Who Moved My Cheese?,"['Business', 'Self Help', 'Nonfiction', 'Psych...",written by spencer johnson ...
3,15287,2,33,4,41929,98,930.0,739326228,9.780739e+12,Memoirs of a Geisha,4.08,1300209.0,['Arthur Golden'],"A literary sensation and runaway bestseller, t...",9.780740e+12,Memoirs of a Geisha,"['Fiction', 'Historical', 'Historical Fiction'...",a literary sensation and runaway...
4,49199,2,301,5,41929,194,4900.0,1892295490,9.781892e+12,Heart of Darkness,3.42,255576.0,['Joseph Conrad'],Dark allegory describes the narrator's journey...,9.781890e+12,Heart of Darkness,"['Classics', 'Fiction', 'Literature', 'Cultura...",dark allegory describes the narr...


Using tutorial from gensim:
https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html

In [10]:
documents = list(temp_df['book_desc_tok'])
documents

['  the    incredible    national    bestseller    that    is    changing    people     s    lives          and    increasing    their    net    worth         can    you    spot    the    millionaire    next    door         who    are    the    rich    in    this    country         what    do    they    do         where    do    they    shop         what    do    they    drive         how    do    they    invest         where    did    their    ancestors    come    from         how    did    they    get    rich         can    i    ever    become    one    of    them         get    the    answers    in    the    millionaire    next    door         the    never before told    story    about    wealth    in    america         you     ll    be    surprised    at    what    you    find    out          ',
 '  an    ingenious    code    hidden    in    the    works    of    leonardo    da    vinci a    desperate    race    through    the    cathedrals    and    castles    of    europe an    a

In [17]:
#lines = []
#for i, line in enumerate(documents):
#    lines.append(gensim.utils.simple_preprocess(line))

#print(lines[0])

['the', 'incredible', 'national', 'bestseller', 'that', 'is', 'changing', 'people', 'lives', 'and', 'increasing', 'their', 'net', 'worth', 'can', 'you', 'spot', 'the', 'millionaire', 'next', 'door', 'who', 'are', 'the', 'rich', 'in', 'this', 'country', 'what', 'do', 'they', 'do', 'where', 'do', 'they', 'shop', 'what', 'do', 'they', 'drive', 'how', 'do', 'they', 'invest', 'where', 'did', 'their', 'ancestors', 'come', 'from', 'how', 'did', 'they', 'get', 'rich', 'can', 'ever', 'become', 'one', 'of', 'them', 'get', 'the', 'answers', 'in', 'the', 'millionaire', 'next', 'door', 'the', 'never', 'before', 'told', 'story', 'about', 'wealth', 'in', 'america', 'you', 'll', 'be', 'surprised', 'at', 'what', 'you', 'find', 'out']


In [8]:
import smart_open

In [18]:
def read_corpus(f, tokens_only=False):
    #with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(documents))

In [19]:
train_corpus

[TaggedDocument(words=['the', 'incredible', 'national', 'bestseller', 'that', 'is', 'changing', 'people', 'lives', 'and', 'increasing', 'their', 'net', 'worth', 'can', 'you', 'spot', 'the', 'millionaire', 'next', 'door', 'who', 'are', 'the', 'rich', 'in', 'this', 'country', 'what', 'do', 'they', 'do', 'where', 'do', 'they', 'shop', 'what', 'do', 'they', 'drive', 'how', 'do', 'they', 'invest', 'where', 'did', 'their', 'ancestors', 'come', 'from', 'how', 'did', 'they', 'get', 'rich', 'can', 'ever', 'become', 'one', 'of', 'them', 'get', 'the', 'answers', 'in', 'the', 'millionaire', 'next', 'door', 'the', 'never', 'before', 'told', 'story', 'about', 'wealth', 'in', 'america', 'you', 'll', 'be', 'surprised', 'at', 'what', 'you', 'find', 'out'], tags=[0]),
 TaggedDocument(words=['an', 'ingenious', 'code', 'hidden', 'in', 'the', 'works', 'of', 'leonardo', 'da', 'vinci', 'desperate', 'race', 'through', 'the', 'cathedrals', 'and', 'castles', 'of', 'europe', 'an', 'astonishing', 'truth', 'concea

To train the model, we’ll need to associate a tag/number with each document of the training corpus. In our case, the tag is simply the zero-based line number.



In [20]:
#epochs?
model = gensim.models.doc2vec.Doc2Vec(vector_size = 768, window=5, min_count = 2, workers = -1, epochs=40)


In [21]:
model.build_vocab(train_corpus)

In [22]:
print(f"Word 'penalty' appeared {model.wv.get_vecattr('penalty', 'count')} times in the training corpus.")


Word 'penalty' appeared 3 times in the training corpus.


Next, train the model on the corpus. If optimized Gensim (with BLAS library) is being used, this should take no more than 3 seconds. If the BLAS library is not being used, this should take no more than 2 minutes, so use optimized Gensim with BLAS if you value your time.

In [23]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)


Now, we can use the trained model to infer a vector for any piece of text by passing a list of words to the model.infer_vector() function. 

In [ ]:
#vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
#print(vector)

Note that infer_vector() does not take a string, but rather a list of string tokens, which should have already been tokenized the same way as the words property of original training document objects.

In [31]:
print(documents[0].split())

['the', 'incredible', 'national', 'bestseller', 'that', 'is', 'changing', 'people', 's', 'lives', 'and', 'increasing', 'their', 'net', 'worth', 'can', 'you', 'spot', 'the', 'millionaire', 'next', 'door', 'who', 'are', 'the', 'rich', 'in', 'this', 'country', 'what', 'do', 'they', 'do', 'where', 'do', 'they', 'shop', 'what', 'do', 'they', 'drive', 'how', 'do', 'they', 'invest', 'where', 'did', 'their', 'ancestors', 'come', 'from', 'how', 'did', 'they', 'get', 'rich', 'can', 'i', 'ever', 'become', 'one', 'of', 'them', 'get', 'the', 'answers', 'in', 'the', 'millionaire', 'next', 'door', 'the', 'never', 'before', 'told', 'story', 'about', 'wealth', 'in', 'america', 'you', 'll', 'be', 'surprised', 'at', 'what', 'you', 'find', 'out']


In [33]:
paragraph_vectors = []

for document in documents:
    paragraph_vectors.append(model.infer_vector(document.split()))

In [36]:
paragraph_vectors[0]

array([-2.66651128e-04,  1.67479899e-04,  1.37491152e-05,  5.35502622e-04,
        2.23769413e-04, -5.15385473e-04, -6.26053254e-04,  1.30902743e-04,
       -4.60880483e-06, -1.07500899e-04,  7.04506747e-05,  4.15283808e-04,
       -5.04231837e-04,  1.61069911e-04,  1.15020666e-05, -2.01045259e-04,
        3.81403770e-05,  4.94669483e-04,  2.26647444e-05, -4.01010038e-04,
       -3.47274290e-05, -5.82919456e-04, -1.94156237e-04,  6.47309062e-04,
       -2.60487577e-04,  2.73530924e-04, -1.73038032e-04, -6.10606861e-04,
        2.41064467e-04,  5.47451957e-04, -1.61739765e-04, -4.70320898e-04,
       -2.57974112e-04,  4.31235094e-04,  5.45218754e-05,  6.18676888e-04,
       -4.57292954e-05, -3.49072739e-04,  1.23681501e-04,  3.38402722e-04,
        5.47159056e-04,  2.99340201e-04,  1.90395556e-04, -5.65332070e-04,
       -3.63760948e-04,  3.98712466e-04, -4.35837079e-04, -2.97860708e-04,
        3.87429114e-04,  1.04983723e-04,  2.86675378e-04,  5.92592805e-05,
       -2.79520260e-04,  

In [37]:
len(paragraph_vectors)

4327

In [38]:
len(paragraph_vectors[0])

768

In [39]:
paragraph_vectors_df = pd.DataFrame(paragraph_vectors)

In [40]:
paragraph_vectors_df.shape

(4327, 768)

In [41]:
paragraph_vectors_df.to_csv('book_embeddings_from_paragraph_vectors.csv')

Also note that because the underlying training/inference algorithms are an iterative approximation problem that makes use of internal randomization, repeated inferences of the same text will return slightly different vectors.

